In [1]:
import numpy as np
import pandas as pd
from itertools import product

from pyomo.environ import *

# Define data
attrs = ['den', 'bnz', 'roz', 'moz']

sources_data = {
    "s1": [49.2, 6097.56, {'den': 0.82, 'bnz':3, 'roz':99.2,'moz':90.5}],
    "s2": [62.0, 16129, {'den': 0.62, 'bnz':0, 'roz':87.9,'moz':83.5}],
    "s3": [300.0, 500, {'den': 0.75, 'bnz':0, 'roz':114,'moz':98.7}]
}

targets_data = {
    "t1": [190, 500, {'den': 0.74, 'roz':95, 'moz':85, 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0}],
    "t2": [230, 500, {'den': 0.74, 'roz':96, 'moz':88, 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0.9}],
    "t3": [150, 500, {'den': 0.74, 'roz':91, 'moz':0 , 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0}]
}

pools_data = {
    "p1": 1250,
    "p2": 1750
}

In [2]:
sources = list(sources_data.keys())
targets = list(targets_data.keys())
pools = list(pools_data.keys())

cost = {s: sources_data[s][0] for s in sources}
supply = {s: sources_data[s][1] for s in sources}
content = {s: sources_data[s][2] for s in sources}

price = {t: targets_data[t][0] for t in targets}
demand = {t: targets_data[t][1] for t in targets}
min_tol = {t: targets_data[t][2] for t in targets}
max_tol = {t: targets_data[t][3] for t in targets}

cap = {p: pools_data[p] for p in pools}

# p_pooling
p_pooling = ConcreteModel()

# Sets
p_pooling.sources = Set(initialize=sources)
p_pooling.targets = Set(initialize=targets)
p_pooling.pools = Set(initialize=pools)
p_pooling.attrs = Set(initialize=attrs)

# Parameters
p_pooling.cost = Param(p_pooling.sources, initialize=cost)
p_pooling.supply = Param(p_pooling.sources, initialize=supply)
p_pooling.content = Param(p_pooling.sources, within=Any, initialize=content)

p_pooling.price = Param(p_pooling.targets, initialize=price)
p_pooling.demand = Param(p_pooling.targets, initialize=demand)
p_pooling.min_tol = Param(p_pooling.targets, within=Any, initialize=min_tol)
p_pooling.max_tol = Param(p_pooling.targets, within=Any, initialize=max_tol)

p_pooling.cap = Param(p_pooling.pools, initialize=cap)

# Decision variables
p_pooling.ik = Var(p_pooling.sources, p_pooling.targets, domain=NonNegativeReals)
p_pooling.ij = Var(p_pooling.sources, p_pooling.pools, domain=NonNegativeReals)
p_pooling.jk = Var(p_pooling.pools, p_pooling.targets, domain=NonNegativeReals)
p_pooling.prop = Var(p_pooling.pools, p_pooling.attrs, domain=NonNegativeReals)

In [3]:
# Constraints
def flow_conservation_rule(model, j):
    return sum(model.ij[i, j] for i in model.sources) == sum(model.jk[j, k] for k in model.targets)

p_pooling.flow_conservation_con = Constraint(p_pooling.pools, rule=flow_conservation_rule)

def source_capacity_rule(model, i):
    return sum(model.ij[i, j] for j in model.pools) + sum(model.ik[i, k] for k in model.targets) <= model.supply[i]

p_pooling.source_capacity_con = Constraint(p_pooling.sources, rule=source_capacity_rule)

def pool_capacity_rule(model, j):
    return sum(model.jk[j, k] for k in model.targets) <= model.cap[j]

p_pooling.pool_capacity_con = Constraint(p_pooling.pools, rule=pool_capacity_rule)

def target_demand_rule(model, k):
    return sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools) >= model.demand[k]

p_pooling.target_demand_con = Constraint(p_pooling.targets, rule=target_demand_rule)

def pool_concentration_rule(model, j, attr):
    return sum(model.content[i][attr] * model.ij[i, j] for i in model.sources) == model.prop[j, attr] * sum(model.jk[j, k] for k in model.targets)

p_pooling.pool_concentration_con = ConstraintList()

for j in pools:
    for attr in attrs:
        p_pooling.pool_concentration_con.add(p_pooling.prop[j, attr] == 0)

def target_tolerance_rule(model, k, attr):
    return sum(model.content[i][attr] * model.ik[i, k] for i in model.sources) + sum(model.prop[j, attr] * model.jk[j, k] for j in model.pools) >= model.min_tol[k][attr] * (sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools))

p_pooling.target_min_tolerance_con = ConstraintList()
p_pooling.target_max_tolerance_con = ConstraintList()
for k in targets:
    for attr in min_tol[k].keys():
        p_pooling.target_min_tolerance_con.add(sum(p_pooling.content[i][attr] * p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.prop[j, attr] * p_pooling.jk[j, k] for j in p_pooling.pools) >= p_pooling.min_tol[k][attr] * (sum(p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.jk[j, k] for j in p_pooling.pools)))
        p_pooling.target_max_tolerance_con.add(sum(p_pooling.content[i][attr] * p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.prop[j, attr] * p_pooling.jk[j, k] for j in p_pooling.pools) <= p_pooling.max_tol[k][attr] * (sum(p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.jk[j, k] for j in p_pooling.pools)))


In [4]:
# Objective function
def total_profit_rule(model):
    return(sum(model.price[k] * 
                    (sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools)) for k in model.targets) - 
           sum(model.cost[i] * 
                    (sum(model.ij[i, j] for j in model.pools) + sum(model.ik[i, k] for k in model.targets)) for i in model.sources)
          )

p_pooling.total_profit_obj = Objective(rule=total_profit_rule, sense=maximize)

In [6]:
# Solve the model
solver = SolverFactory('gurobi')
results = solver.solve(p_pooling)

# Print results
if results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found!")
    print("Objective Value:", value(p_pooling.total_profit_obj))
else:
    print("Solver terminated with status:", results.solver.status)

model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem proven to be infeasible or unbounded.
Solver terminated with status: warning
